In [ ]:
# Importation de modules

import re
# Importation du module qui permet d'écrire des Regex
import os
# Importation du module qui permet de modifier des fichiers sur le système d'exploitation
import csv as csvlib
# Importation du module permettant de lire et d'écrire du .csv
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# Importation des modules permettant de comparer des chaînes de caractères

In [1]:
# initier un compteur de phrases
id_phrase = 0

# créer un dictionnaire csv_nodes[id_text + "ph" + id_phrase] = [id_text, id_phrase, text, label]
csv_nodes = {}
# créer un dictionnaire csv_edges[source] = [target, lev_distance]
csv_edges = {}

for root, dirs, files in os.walk("mazarinades", topdown=False):
    for name in files:
        print(os.path.join(root, name))
# pour chaque dossier du dossier mazarinades
      # pour chaque fichier .xml de chaque dossier
        # id_phrase = 0
        # récupérer le titre dans label
        # récupérer l'identifiant dans id_text
        # csv_nodes[id_text + "ph" + id_phrase] = [id_text, id_phrase, "null", label]
        # récupérer le texte dans full_text
        # dans full-text, supprimer les "¬" et les retours à la ligne <lb/>
        # dans full_text, tout mettre en minuscules
        # dans full_text, supprimer les diacritiques
        # phrases = full_text.split(".")
        # for phrase in phrases:
        # id_phrase += 1
        # csv_nodes[id_text + "ph" + id_phrase] = [id_text, id_phrase, phrase, "null"]

# créer une liste de textes déjà évalués analysed_texts = []

# for id, node in csv_nodes.items():
      # analysed_texts.append(node[0])
      # for cur_id, cur_node in csv_nodes.items():
        # lev_distance = fuzz.ratio(node[2], cur_node[2]) / 100
        # if (lev_distance >= 0.80
        # && cur_node[0] not in analysed_texts
        # && node[2] != "null"
        # && cur_node[2] != "null"):
        # csv_edges[id] = [cur_id, lev_distance]


# remplir les dictionnaires
# verser le contenu dans un fichier maz_phrases.csv


mazarinades/2101-2200/Moreau2157_GBOOKS.xml
mazarinades/2101-2200/Moreau2160_GBOOKS.xml
mazarinades/2101-2200/Moreau2132_GALL.xml
mazarinades/2101-2200/Moreau2188_MAZ.xml
mazarinades/2101-2200/Moreau2129_GBOOKS.xml
mazarinades/2101-2200/Moreau2113_GBOOKS.xml
mazarinades/2101-2200/Moreau2146_GBOOKS.xml
mazarinades/2101-2200/Moreau2168_GBOOKS.xml
mazarinades/2101-2200/Moreau2116_GBOOKS.xml
mazarinades/2101-2200/Moreau2143_GBOOKS.xml
mazarinades/2101-2200/Moreau2164_MAZ.xml
mazarinades/2101-2200/Moreau2107_GBOOKS.xml
mazarinades/2101-2200/Moreau2130_GBOOKS.xml
mazarinades/2101-2200/Moreau2149_GBOOKS.xml
mazarinades/2101-2200/Moreau2181_GBOOKS.xml
mazarinades/2101-2200/Moreau2155_GBOOKS.xml
mazarinades/2101-2200/Moreau2126_GBOOKS.xml
mazarinades/2101-2200/Moreau2111_GBOOKS.xml
mazarinades/2101-2200/Moreau2140_MAZ.xml
mazarinades/2101-2200/Moreau2144_GBOOKS.xml
mazarinades/2101-2200/Moreau2173_GBOOKS.xml
mazarinades/2101-2200/Moreau2158_GBOOKS.xml
mazarinades/2101-2200/Moreau2114_GBOOKS.xml